## Imports

In [2]:
import re
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## Utils

In [ ]:
def parse_data(text):
    pattern = re.compile(
        r"Algorithm:\s*(?P<Algorithm>\S+)\s*"
        r"Instance:\s*(?P<Instance>\S+)\s*"
        r"Runtime:\s*(?P<Runtime>\d+)\s*nanoseconds\s*"
        r"Score:\s*(?P<Score>\d+)\s*"
        r"Number\s*of\s*Evaluations:\s*(?P<Number_of_Evaluations>\d+)\s*"
        r"Number\s*of\s*Performed\s*Moves:\s*(?P<Number_of_Performed_Moves>\d+)\s*"
        r"Solution:\s*(?P<Solution>[\d\s]+)\s*"
        r"Optimal\s*Score:\s*(?P<Optimal_Score>\d+)\s*"
        r"Optimal\s*Solution:\s*(?P<Optimal_Solution>[\d\s]+)"
    )
    
    rows = []
    for match in pattern.finditer(text):
        row = match.groupdict()
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    numerical_cols = ['Runtime', 'Score', 'Number_of_Evaluations', 'Number_of_Performed_Moves', 'Optimal_Score']
    df[numerical_cols] = df[numerical_cols].apply(pd.to_numeric)
    
    return df

## Read Data

In [5]:
with open('new_results.txt', 'r') as file:
    data = file.read()

df = parse_data(data)

In [7]:
df.head()

,Algorithm,Instance,Runtime,Score,Number_of_Evaluations,Number_of_Performed_Moves,Solution,Optimal_Score,Optimal_Solution
0,heuristic,bur26a,4253,5841774,1,1,16 14 10 25 9 21 12 24 18 1 20 6 13 17 11 0 3 ...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
1,random,bur26a,418,6006721,1,1,7 11 5 21 14 22 1 4 16 3 13 2 9 23 25 8 6 15 2...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
2,antiheuristic,bur26a,4593,6013250,1,1,1 10 14 21 3 25 11 20 23 16 24 13 6 22 12 7 9 ...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
3,heuristic,bur26a,4197,5841774,1,1,16 14 10 25 9 21 12 24 18 1 20 6 13 17 11 0 3 ...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...
4,greedy_LS,bur26a,147376,5436985,39921,78,3 1 10 11 6 2 25 23 14 0 18 19 8 17 12 15 13 2...,5426670,11 15 26 7 4 13 12 2 6 18 9 5 1 21 8 14 3 20 1...


## Analysis